In [4]:
import pandas as pd
import numpy as np
import plotly.express as px

# CSV読み込み
df = pd.read_csv("earthquakes.csv")

# 日本時間へ変換
df["time"] = pd.to_datetime(df["time"])
df["time_jst"] = df["time"] + pd.Timedelta(hours=9)

# 3時間ごとに丸める
df_anim = df.copy()
df_anim["time_3h"] = df_anim["time_jst"].dt.floor("3H")
df_anim["time_str"] = df_anim["time_3h"].dt.strftime("%Y-%m-%d %H:00")

# 深さで色分け
conditions = [
    (df_anim["depth"] <= 70),
    (df_anim["depth"] > 70) & (df_anim["depth"] <= 300),
    (df_anim["depth"] > 300)
]

colors = ["green", "orange", "blue"]  # 浅・中・深

df_anim["depth_color"] = np.select(conditions, colors, default="gray")

# マグニチュードでサイズ
df_anim["size"] = df_anim["mag"] ** 2


C:\Users\asami\AppData\Local\Temp\ipykernel_3540\2687732498.py:14: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_anim["time_3h"] = df_anim["time_jst"].dt.floor("3H")


In [62]:
import pandas as pd
import requests

# USGSから過去1か月の地震データを取得
url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_month.csv"
response = requests.get(url)

# データをファイルに保存
with open('earthquakes.csv', 'wb') as file:
    file.write(response.content)

# データをDataFrameに読み込む
data = pd.read_csv('earthquakes.csv')

# データの最初の5行を表示
data.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2025-11-19T02:14:34.839Z,59.385216,-138.712936,5.815022,2.769645,ml,15.0,186.461739,0.450602,0.807060,...,2025-11-19T02:16:45.431Z,"60 km ESE of Yakutat, Alaska",earthquake,0.000000,2.748861,0.000000,1.0,automatic,ak,ak
1,2025-11-19T02:08:16.932Z,62.943947,-150.472092,103.293152,2.039515,ml,25.0,45.500549,0.441669,0.488629,...,2025-11-19T02:09:41.617Z,"52 km NNE of Petersville, Alaska",earthquake,0.000000,3.418342,0.479709,2.0,automatic,ak,ak
2,2025-11-19T01:52:37.540Z,37.762333,-121.943001,6.170000,1.790000,md,14.0,94.000000,0.025990,0.110000,...,2025-11-19T02:17:18.670Z,"4 km ESE of San Ramon, CA",earthquake,0.340000,0.660000,0.120000,12.0,automatic,nc,nc
3,2025-11-19T01:48:36.220Z,37.286167,-121.658501,5.660000,2.960000,md,89.0,41.000000,0.029530,0.070000,...,2025-11-19T02:30:16.668Z,"16 km E of Seven Trees, CA",earthquake,0.110000,0.230000,0.150000,132.0,automatic,nc,nc
4,2025-11-19T01:38:54.976Z,31.658000,-104.322000,7.150900,1.700000,ml,31.0,68.000000,0.100000,0.100000,...,2025-11-19T02:17:10.155Z,"57 km S of Whites City, New Mexico",earthquake,0.622503,0.725843,0.100000,18.0,reviewed,tx,tx


In [63]:
# Jupyter 上で実行
!pip install pandas requests plotly



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [64]:
import pandas as pd
import requests
from io import StringIO
from datetime import datetime, timedelta

# ------------------
# 期間を設定（過去1年）
# ------------------
end = datetime.utcnow()
start = end - timedelta(days=365)

# 月ごとに分割
def month_ranges(start, end):
    ranges = []
    cur = start.replace(day=1)
    while cur < end:
        if cur.month == 12:
            nxt = cur.replace(year=cur.year+1, month=1, day=1)
        else:
            nxt = cur.replace(month=cur.month+1, day=1)
        ranges.append((cur, min(nxt, end)))
        cur = nxt
    return ranges

ranges = month_ranges(start, end)

# ------------------
# USGS API
# ------------------
url = "https://earthquake.usgs.gov/fdsnws/event/1/query"

all_df = []

print("Requesting monthly USGS data...")

for s, e in ranges:
    params = {
        "format": "csv",
        "starttime": s.strftime("%Y-%m-%d"),
        "endtime": e.strftime("%Y-%m-%d"),
        "minmagnitude": 2.5,
        "orderby": "time"
    }
    print(f" → {params['starttime']} ～ {params['endtime']}")

    resp = requests.get(url, params=params)
    resp.raise_for_status()  # ← 500 が出たらここで止まる

    df = pd.read_csv(StringIO(resp.text))
    all_df.append(df)

# ------------------
# データを結合
# ------------------
df_all = pd.concat(all_df, ignore_index=True)

print(f"\nDONE! 総イベント数: {len(df_all)} 件")

# ------------------
# 日本時間（JST）に変換
# ------------------
df_all["time_jst"] = pd.to_datetime(df_all["time"], utc=True).dt.tz_convert("Asia/Tokyo")

df_all.head()


C:\Users\asami\AppData\Local\Temp\ipykernel_3540\101846607.py:9: DeprecationWarning:

datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).



Requesting monthly USGS data...
 → 2024-11-01 ～ 2024-12-01
 → 2024-12-01 ～ 2025-01-01
 → 2025-01-01 ～ 2025-02-01
 → 2025-02-01 ～ 2025-03-01
 → 2025-03-01 ～ 2025-04-01
 → 2025-04-01 ～ 2025-05-01
 → 2025-05-01 ～ 2025-06-01
 → 2025-06-01 ～ 2025-07-01
 → 2025-07-01 ～ 2025-08-01
 → 2025-08-01 ～ 2025-09-01
 → 2025-09-01 ～ 2025-10-01
 → 2025-10-01 ～ 2025-11-01
 → 2025-11-01 ～ 2025-11-19

DONE! 総イベント数: 29291 件


,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource,time_jst
0,2024-11-30T23:52:45.796Z,-7.657000,127.755500,168.706,4.20,mb,19.0,91.0,3.777000,0.93,...,"126 km NE of Lospalos, Timor Leste",earthquake,8.64,8.252,0.132000,16.0,reviewed,us,us,2024-12-01 08:52:45.796000+09:00
1,2024-11-30T23:19:38.580Z,19.386167,-155.242833,1.960,3.08,ml,28.0,67.0,0.008029,0.11,...,"6 km S of Volcano, Hawaii",earthquake,0.17,0.150,0.397188,22.0,reviewed,hv,hv,2024-12-01 08:19:38.580000+09:00
2,2024-11-30T23:18:42.830Z,19.387667,-155.242833,1.810,2.50,ml,49.0,52.0,0.008285,0.10,...,"6 km S of Volcano, Hawaii",earthquake,0.19,0.180,0.262802,32.0,reviewed,hv,hv,2024-12-01 08:18:42.830000+09:00
3,2024-11-30T22:12:26.968Z,-22.581700,-12.956900,10.000,4.80,mb,42.0,50.0,26.676000,1.01,...,southern Mid-Atlantic Ridge,earthquake,12.03,1.884,0.089000,39.0,reviewed,us,us,2024-12-01 07:12:26.968000+09:00
4,2024-11-30T21:36:05.908Z,13.054300,143.926400,132.912,4.70,mb,150.0,40.0,1.060000,0.66,...,"83 km WSW of Merizo Village, Guam",earthquake,9.23,5.913,0.049000,129.0,reviewed,us,us,2024-12-01 06:36:05.908000+09:00


In [110]:
import requests
import pandas as pd
from io import StringIO
from datetime import datetime, timedelta, timezone

# ======================================
# ① 取得期間を設定（過去7日間）
# ======================================
end_time = datetime.now(timezone.utc)
start_time = end_time - timedelta(days=7)

# USGS API
url = "https://earthquake.usgs.gov/fdsnws/event/1/query"
params = {
    "format": "csv",
    "starttime": start_time.strftime("%Y-%m-%dT%H:%M:%S"),
    "endtime": end_time.strftime("%Y-%m-%dT%H:%M:%S"),
    "minmagnitude": 2.5
}

print("Requesting USGS data ...")
resp = requests.get(url, params=params)
resp.raise_for_status()

# CSV を pandas に読み込む
df = pd.read_csv(StringIO(resp.text))
print(f"Raw events fetched: {len(df)}")

# query.csv に保存
df.to_csv("query.csv", index=False)
print("Saved to query.csv")


Requesting USGS data ...
Raw events fetched: 247
Saved to query.csv


In [111]:
import pandas as pd

# query.csv を読み込み
df = pd.read_csv("query.csv")
print("CSV loaded:", df.shape)

# UTC → JST に変換
df["time_jst"] = (
    pd.to_datetime(df["time"], utc=True)
      .dt.tz_convert("Asia/Tokyo")
)

print(df[["time", "time_jst", "latitude", "longitude", "mag"]].head())


CSV loaded: (247, 22)
                       time                         time_jst   latitude  \
0  2025-11-19T02:45:00.925Z 2025-11-19 11:45:00.925000+09:00  55.460300   
1  2025-11-19T02:23:19.189Z 2025-11-19 11:23:19.189000+09:00  39.235200   
2  2025-11-19T02:14:34.839Z 2025-11-19 11:14:34.839000+09:00  59.385216   
3  2025-11-19T01:48:36.220Z 2025-11-19 10:48:36.220000+09:00  37.286167   
4  2025-11-19T01:38:53.994Z 2025-11-19 10:38:53.994000+09:00  10.870900   

    longitude       mag  
0 -157.214157  2.547544  
1  143.467800  5.000000  
2 -138.712936  2.769645  
3 -121.658501  2.960000  
4  -62.756600  5.000000  


In [112]:
pip install pandas plotly requests


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [117]:
import numpy as np

# 深さカテゴリ
conditions = [
    (df["depth"] < 10),
    (df["depth"] >= 10) & (df["depth"] < 50),
    (df["depth"] >= 50)
]

choices = ["浅い(Shallow)", "中間(Middle)", "深い(Deep)"]

df["depth_category"] = np.select(
    conditions,
    choices,
    default="不明(Unknown)"
)


import json

records = df.to_dict(orient="records")

with open("earthquake_data.json", "w", encoding="utf-8") as f:
    json.dump(records, f, ensure_ascii=False)

import plotly.express as px

fig = px.scatter_geo(
    df,
    lon="longitude",
    lat="latitude",
    color="depth_category",
    size="mag",
    animation_frame="date_jst",
    hover_name="time_jst",
    projection="natural earth",
    title="過去30日間の地震アニメーション（JST）"
)

fig.update_layout(
    legend_title="深さカテゴリ",
    geo=dict(
        showland=True,
        landcolor="rgb(229, 229, 229)"
    )
)

fig.show()



TypeError: Object of type Timestamp is not JSON serializable

In [118]:
import json
import numpy as np

# ① JST を文字列化（datetime は JSON にできない）
df["time"] = df["time"].astype(str)
df["time_jst"] = df["time_jst"].astype(str)
df["date_jst"] = df["date_jst"].astype(str)

# ② numpy 型 → Python ネイティブ型に変換
df = df.astype({
    "latitude": float,
    "longitude": float,
    "depth": float,
    "mag": float
})

# カテゴリも文字列確定
df["depth_category"] = df["depth_category"].astype(str)

# ③ 辞書化 → JSON 書き出し
records = df.to_dict(orient="records")

with open("earthquake_data.json", "w", encoding="utf-8") as f:
    json.dump(records, f, ensure_ascii=False)


KeyError: 'date_jst'

In [134]:
import pandas as pd

# CSV を読み込み
df = pd.read_csv("query.csv")
print("CSV loaded:", df.shape)

# UTC → JST に変換
df["time_jst"] = (
    pd.to_datetime(df["time"], utc=True)
      .dt.tz_convert("Asia/Tokyo")
)

# ← これが必要！
df["date_jst"] = df["time_jst"].dt.date

print(df[["time", "time_jst", "date_jst", "latitude", "longitude", "mag"]].head())


CSV loaded: (247, 22)
                       time                         time_jst    date_jst  \
0  2025-11-19T02:45:00.925Z 2025-11-19 11:45:00.925000+09:00  2025-11-19   
1  2025-11-19T02:23:19.189Z 2025-11-19 11:23:19.189000+09:00  2025-11-19   
2  2025-11-19T02:14:34.839Z 2025-11-19 11:14:34.839000+09:00  2025-11-19   
3  2025-11-19T01:48:36.220Z 2025-11-19 10:48:36.220000+09:00  2025-11-19   
4  2025-11-19T01:38:53.994Z 2025-11-19 10:38:53.994000+09:00  2025-11-19   

    latitude   longitude       mag  
0  55.460300 -157.214157  2.547544  
1  39.235200  143.467800  5.000000  
2  59.385216 -138.712936  2.769645  
3  37.286167 -121.658501  2.960000  
4  10.870900  -62.756600  5.000000  


In [135]:
import folium
from folium.plugins import MarkerCluster

# 地図の中心（適当に日本周辺に設定、必要あれば変更）
m = folium.Map(location=[35, 140], zoom_start=3)

# MarkerCluster（点が多いのでまとめる）
cluster = MarkerCluster().add_to(m)

# 地震をマップに表示
for _, row in df.iterrows():
    lat = row["latitude"]
    lon = row["longitude"]
    mag = row["mag"]
    time = row["time_jst"]

    folium.CircleMarker(
        location=[lat, lon],
        radius=max(mag, 1),  # マグニチュードで大きさ変える
        color="red",
        fill=True,
        fill_opacity=0.7,
        popup=f"M {mag}<br>{time}"
    ).add_to(cluster)

# HTMLに保存
m.save("earthquakes_map.html")
print("✔ 地図を保存しました → earthquakes_map.html")


✔ 地図を保存しました → earthquakes_map.html


In [137]:
import folium
from folium.plugins import MarkerCluster
import webbrowser
import os

# 地図の中心
m = folium.Map(location=[35, 140], zoom_start=3)

# マーカーをまとめる
cluster = MarkerCluster().add_to(m)

# 地震をマップに表示
for _, row in df.iterrows():
    lat = row["latitude"]
    lon = row["longitude"]
    mag = row["mag"]
    time = row["time_jst"]

    folium.CircleMarker(
        location=[lat, lon],
        radius=max(mag, 1),
        color="red",
        fill=True,
        fill_opacity=0.7,
        popup=f"M {mag}<br>{time}"
    ).add_to(cluster)

# 保存
filename = "earthquakes_map.html"
m.save(filename)
print("✔ 地図ファイル作成:", filename)

# ブラウザで自動オープン
fullpath = os.path.abspath(filename)
webbrowser.open(f"file://{fullpath}")
print("🌏 自動でブラウザを開きました")



✔ 地図ファイル作成: earthquakes_map.html
🌏 自動でブラウザを開きました


In [138]:
import pandas as pd
import plotly.express as px

# -----------------------------
# 1) データ読み込み（USGS 過去1年）
# -----------------------------
url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_year.csv"
df = pd.read_csv(url)

# -----------------------------
# 2) UTC → JST
# -----------------------------
df["time_jst"] = (
    pd.to_datetime(df["time"], utc=True)
      .dt.tz_convert("Asia/Tokyo")
)

# -----------------------------
# 3) 3時間ごとに丸める
# -----------------------------
df["time_bin"] = df["time_jst"].dt.floor("3H")

# -----------------------------
# 4) Plotly アニメーション地図
# -----------------------------
fig = px.scatter_geo(
    df,
    lat="latitude",
    lon="longitude",
    color="mag",
    size="mag",
    hover_name="place",
    animation_frame="time_bin",
    projection="natural earth",
    color_continuous_scale="Turbo",
    range_color=(0, df["mag"].max()),
    title="🌏 過去1年間の地震（3時間ごとのアニメーション, JST）"
)

fig.update_layout(
    height=700,
    geo=dict(
        showland=True,
        landcolor="rgb(230,230,230)",
        showocean=True,
        oceancolor="rgb(200,220,255)"
    )
)

fig.show()


KeyError: 'time'

In [139]:
import pandas as pd

url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_year.csv"
df = pd.read_csv(url)

print(df.columns.tolist())


['404 File Not Found']


In [140]:
import pandas as pd

# ▼ 1年間のデータ取得（今日基準）
end = pd.Timestamp.utcnow()
start = end - pd.Timedelta(days=365)

url = (
    "https://earthquake.usgs.gov/fdsnws/event/1/query"
    f"?format=csv&starttime={start.strftime('%Y-%m-%d')}"
    f"&endtime={end.strftime('%Y-%m-%d')}"
)

df = pd.read_csv(url)
print("Loaded:", df.shape)

# ▼ UTC → JST
df["time"] = pd.to_datetime(df["time"], utc=True)
df["time_jst"] = df["time"].dt.tz_convert("Asia/Tokyo")
df["date_jst"] = df["time_jst"].dt.floor("3h")   # 3時間ごとに丸める

df.head()


HTTPError: HTTP Error 400: Bad Request

In [142]:
import pandas as pd
import datetime as dt

end = dt.datetime.utcnow()
start = end - dt.timedelta(days=365)

url = (
    "https://earthquake.usgs.gov/fdsnws/event/1/query"
    f"?format=csv"
    f"&starttime={start.strftime('%Y-%m-%d')}"
    f"&endtime={end.strftime('%Y-%m-%d')}"
    f"&orderby=time"
    f"&limit=20000"
)

print(url)

df = pd.read_csv(url)
print("Loaded:", df.shape)


C:\Users\asami\AppData\Local\Temp\ipykernel_3540\2411888538.py:4: DeprecationWarning:

datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).



https://earthquake.usgs.gov/fdsnws/event/1/query?format=csv&starttime=2024-11-19&endtime=2025-11-19&orderby=time&limit=20000
Loaded: (20000, 22)


In [143]:
import pandas as pd
import datetime as dt

def fetch_usgs_month(start, end):
    """1ヶ月分の USGS 地震データ(最大20万件) を CSV で取得"""
    url = (
        "https://earthquake.usgs.gov/fdsnws/event/1/query"
        f"?format=csv"
        f"&starttime={start.strftime('%Y-%m-%d')}"
        f"&endtime={end.strftime('%Y-%m-%d')}"
        f"&limit=20000"
        f"&orderby=time"
    )
    print("Downloading:", url)
    return pd.read_csv(url)


# ▼ 過去1年の区間を月ごとに分割して取得
end = dt.datetime.utcnow()
start = end - dt.timedelta(days=365)

dfs = []
cur = start

while cur < end:
    # 月末の計算
    next_month = (cur.replace(day=28) + dt.timedelta(days=4)).replace(day=1)
    month_end = min(next_month, end)

    try:
        df_month = fetch_usgs_month(cur, month_end)
        dfs.append(df_month)
        print(f"  → {len(df_month)} rows")
    except Exception as e:
        print("Error:", e)

    cur = month_end  # 次の月へ

# ▼ 全部結合
df_all = pd.concat(dfs, ignore_index=True)
print("\n=== DONE ===")
print("Total rows:", len(df_all))

# ▼ UTC → JST変換（必要なら）
df_all["time"] = pd.to_datetime(df_all["time"]) + pd.Timedelta(hours=9)
df_all.rename(columns={"time": "time_jst"}, inplace=True)

df_all.head()


C:\Users\asami\AppData\Local\Temp\ipykernel_3540\2344315134.py:19: DeprecationWarning:

datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).



Downloading: https://earthquake.usgs.gov/fdsnws/event/1/query?format=csv&starttime=2024-11-19&endtime=2024-12-01&limit=20000&orderby=time
  → 4284 rows
Downloading: https://earthquake.usgs.gov/fdsnws/event/1/query?format=csv&starttime=2024-12-01&endtime=2025-01-01&limit=20000&orderby=time
  → 13050 rows
Downloading: https://earthquake.usgs.gov/fdsnws/event/1/query?format=csv&starttime=2025-01-01&endtime=2025-02-01&limit=20000&orderby=time
  → 11642 rows
Downloading: https://earthquake.usgs.gov/fdsnws/event/1/query?format=csv&starttime=2025-02-01&endtime=2025-03-01&limit=20000&orderby=time
  → 11489 rows
Downloading: https://earthquake.usgs.gov/fdsnws/event/1/query?format=csv&starttime=2025-03-01&endtime=2025-04-01&limit=20000&orderby=time
  → 11863 rows
Downloading: https://earthquake.usgs.gov/fdsnws/event/1/query?format=csv&starttime=2025-04-01&endtime=2025-05-01&limit=20000&orderby=time
  → 11234 rows
Downloading: https://earthquake.usgs.gov/fdsnws/event/1/query?format=csv&starttime=

,time_jst,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2024-12-01 08:54:01.872000+00:00,56.217500,-157.973100,61.800,1.90,ml,NaN,NaN,NaN,0.40,...,2025-01-07T21:33:55.459Z,"28 km ESE of Chignik, Alaska",earthquake,NaN,0.400,NaN,NaN,reviewed,ak,ak
1,2024-12-01 08:52:45.796000+00:00,-7.657000,127.755500,168.706,4.20,mb,19.0,91.0,3.777000,0.93,...,2025-02-02T00:54:22.040Z,"126 km NE of Lospalos, Timor Leste",earthquake,8.64,8.252,0.132000,16.0,reviewed,us,us
2,2024-12-01 08:52:23.360000+00:00,38.825001,-122.801498,3.010,0.71,md,11.0,71.0,0.006888,0.05,...,2024-12-01T00:17:18.431Z,"7 km NNW of The Geysers, CA",earthquake,0.48,0.830,0.280000,12.0,automatic,nc,nc
3,2024-12-01 08:49:23.160000+00:00,35.536833,-96.754000,5.560,-1.94,ml,8.0,140.0,0.013497,0.01,...,2024-12-03T18:39:49.346Z,"8 km NW of Prague, Oklahoma",earthquake,NaN,0.100,0.000000,1.0,reviewed,ok,ok
4,2024-12-01 08:46:06.360000+00:00,60.659667,-152.569667,22.400,1.11,ml,3.0,341.0,0.182100,0.06,...,2024-12-04T20:05:20.540Z,"68 km W of Salamatof, Alaska",earthquake,3.12,3.090,0.356026,4.0,reviewed,av,av


In [144]:
pip install folium pandas branca



SyntaxError: invalid syntax (1882062539.py, line 1)

In [145]:
import folium
from folium.plugins import TimestampedGeoJson
import pandas as pd
import webbrowser
import json
from datetime import datetime

# ------------------------------
# df_all に time_jst, latitude, longitude, mag が入っている前提
# ------------------------------

# ▼ 3時間ごとに丸める
df_all["time_floor"] = df_all["time_jst"].dt.floor("3H")

# ▼ Folium で扱える GeoJSON フォーマットに変換
features = []

for _, row in df_all.iterrows():
    feature = {
        "type": "Feature",
        "geometry": {
            "type": "Point",
            "coordinates": [row["longitude"], row["latitude"]],
        },
        "properties": {
            "time": row["time_floor"].strftime("%Y-%m-%dT%H:%M:%S"),
            "popup": f"M{row['mag']}<br>{row['place']}",
            "icon": "circle",
            "iconstyle": {
                "fillColor": "red",
                "fillOpacity": 0.6,
                "stroke": "true",
                "radius": 3 + row["mag"] * 1.5,
            },
        },
    }
    features.append(feature)

geojson = {
    "type": "FeatureCollection",
    "features": features,
}

# ▼ 地図の中心は平均値で良い
center_lat = df_all["latitude"].mean()
center_lon = df_all["longitude"].mean()

# ▼ ベースマップ
m = folium.Map(location=[center_lat, center_lon], zoom_start=3)

# ▼ アニメーションレイヤーを追加
TimestampedGeoJson(
    geojson,
    period="PT3H",
    add_last_point=True,
    auto_play=False,
    loop=True,
    max_speed=1,
    loop_button=True,
    date_options="YYYY-MM-DD HH:mm",
    time_slider_drag_update=True,
).add_to(m)

# ▼ 保存
html_path = "earthquake_animation.html"
m.save(html_path)

# ▼ ブラウザで自動オープン
webbrowser.open(html_path)

print("👉 完了！ブラウザにアニメーションが表示されます。")


C:\Users\asami\AppData\Local\Temp\ipykernel_3540\1044985966.py:13: FutureWarning:

'H' is deprecated and will be removed in a future version, please use 'h' instead.



👉 完了！ブラウザにアニメーションが表示されます。


In [148]:
import folium
from folium.plugins import TimestampedGeoJson
import pandas as pd
import webbrowser
import json

# -------------------------------------------------------
# ▼ df_all が存在する前提（time_jst が datetime）
# -------------------------------------------------------

# ▼ 3時間ごとに丸める
df_all["time_floor"] = df_all["time_jst"].dt.floor("3H")

# ▼ マグニチュードで 3 分割（quantile）
df_all["group"] = pd.qcut(df_all["mag"], 3, labels=[0, 1, 2])

# 色設定（3分割）
group_colors = {
    0: "blue",     # 弱い
    1: "yellow",   # 中間
    2: "red",      # 強い
}

# ▼ GeoJSON を構築
features = []

for _, row in df_all.iterrows():
    color = group_colors[int(row["group"])]
    
    feature = {
        "type": "Feature",
        "geometry": {
            "type": "Point",
            "coordinates": [row["longitude"], row["latitude"]],
        },
        "properties": {
            "time": row["time_floor"].strftime("%Y-%m-%dT%H:%M:%S"),
            "popup": f"M{row['mag']} / {row['place']}",
            "icon": "circle",
            "iconstyle": {
                "fillColor": color,
                "fillOpacity": 0.7,
                "stroke": "true",
                "color": color,
                "radius": 4 + row["mag"] * 1.1,
            },
        },
    }
    features.append(feature)

geojson = {
    "type": "FeatureCollection",
    "features": features,
}

# ▼ 地図中央を平均位置に
center_lat = df_all["latitude"].mean()
center_lon = df_all["longitude"].mean()

m = folium.Map(location=[center_lat, center_lon], zoom_start=3)

# ▼ アニメーションレイヤー
TimestampedGeoJson(
    geojson,
    period="PT3H",  
    add_last_point=True,
    auto_play=False,
    loop=True,
    max_speed=1,
    loop_button=True,
    date_options="YYYY-MM-DD HH:mm",
    time_slider_drag_update=True,
).add_to(m)

# ▼ 保存 & 自動オープン
html_path = "earthquake_color3_animation.html"
m.save(html_path)
webbrowser.open(html_path)

print("👉 完了！3グループ色分けアニメーション地図をブラウザで開きました。")


C:\Users\asami\AppData\Local\Temp\ipykernel_3540\4027554516.py:12: FutureWarning:

'H' is deprecated and will be removed in a future version, please use 'h' instead.



👉 完了！3グループ色分けアニメーション地図をブラウザで開きました。


In [147]:
# ▼ mag の NaN があれば除去
df_all = df_all.dropna(subset=["mag"]).copy()

# ▼ 3分割
df_all["group"] = pd.qcut(
    df_all["mag"],
    3,
    labels=[0, 1, 2],
    duplicates="drop"
)

# ▼ NaN の group を除去
df_all = df_all.dropna(subset=["group"]).copy()

# ▼ int に変換
df_all["group"] = df_all["group"].astype(int)


In [149]:
import folium
from folium.plugins import TimestampedGeoJson
import pandas as pd
import webbrowser
import json
import time

# ---------------------------------------------------------
#  ■ サンプルデータ（3時間ごと）
#     → 本来はここにユーザーが取得した1年分のデータを入れる
# ---------------------------------------------------------
data = [
    {"lat": 35.0, "lon": 135.0, "time": "2025-01-01T00:00:00"},
    {"lat": 35.2, "lon": 135.1, "time": "2025-01-01T03:00:00"},
    {"lat": 35.3, "lon": 135.3, "time": "2025-01-01T06:00:00"},
    {"lat": 35.4, "lon": 135.35, "time": "2025-01-01T09:00:00"},
    {"lat": 35.5, "lon": 135.4, "time": "2025-01-01T12:00:00"},
    {"lat": 35.6, "lon": 135.5, "time": "2025-01-01T15:00:00"},
]

df = pd.DataFrame(data)

# ---------------------------------------------------------
#  ■ 3分割して色分け（例：緯度で 3 区画）
# ---------------------------------------------------------
def get_color(lat):
    if lat < 35.25:
        return "rgba(255,0,0,0.4)"      # 赤（透明）
    elif lat < 35.45:
        return "rgba(0,255,0,0.4)"      # 緑（透明）
    else:
        return "rgba(0,0,255,0.4)"      # 青（透明）

df["color"] = df["lat"].apply(get_color)

# ---------------------------------------------------------
#  ■ GeoJSON 形式に変換（TimeDimension 用）
#     12時間（4スライド）保持 → それ以降は自動消去
# ---------------------------------------------------------

features = []

for _, row in df.iterrows():
    feature = {
        "type": "Feature",
        "geometry": {
            "type": "Point",
            "coordinates": [row["lon"], row["lat"]],
        },
        "properties": {
            "times": [row["time"]],
            "style": {
                "color": row["color"],
                "fillColor": row["color"],
                "fillOpacity": 0.4,
                "radius": 9
            },
            "icon": "circle"
        }
    }
    features.append(feature)

geojson = {
    "type": "FeatureCollection",
    "features": features
}

# ---------------------------------------------------------
#  ■ 地図作成
# ---------------------------------------------------------

m = folium.Map(location=[35.3, 135.3], zoom_start=7)

TimestampedGeoJson(
    data=geojson,
    period="PT3H",              # 3時間ステップ
    duration="PT12H",           # 12時間表示(その後消える)
    auto_play=True,
    loop=True,
    add_last_point=True
).add_to(m)

# ---------------------------------------------------------
#  ■ HTML 保存 & ブラウザで自動表示
# ---------------------------------------------------------

html_file = "animated_map.html"
m.save(html_file)

# 自動でブラウザを開く
time.sleep(0.5)
webbrowser.open(html_file)


True

In [150]:
import folium
from folium.plugins import TimestampedGeoJson
import pandas as pd
import numpy as np
import json
import time
import webbrowser

# ---------------------------------------------------------
# ■ 半年間の3時間ごとのデータを生成
# ---------------------------------------------------------

start = pd.Timestamp("2025-01-01 00:00")
end   = pd.Timestamp("2025-06-30 21:00")

times = pd.date_range(start, end, freq="3H")

# サンプルとして、緯度・経度をランダム移動させたデータを生成
# ※ 本来はここに実データを入れる
lats = 35 + np.sin(np.linspace(0, 10, len(times))) * 0.8
lons = 135 + np.cos(np.linspace(0, 10, len(times))) * 0.8

df = pd.DataFrame({
    "time": times,
    "lat": lats,
    "lon": lons
})

# ---------------------------------------------------------
# ■ 緯度で3分割して色を付ける
# ---------------------------------------------------------
def get_color(lat):
    if lat < 35.2:
        return "rgba(255,0,0,0.4)"      # 赤
    elif lat < 35.5:
        return "rgba(0,255,0,0.4)"      # 緑
    else:
        return "rgba(0,0,255,0.4)"      # 青

df["color"] = df["lat"].apply(get_color)

# ---------------------------------------------------------
# ■ TimeDimension 用 GeoJSON へ変換
#    duration="PT12H" → 半日で点を消す
# ---------------------------------------------------------

features = []
for _, row in df.iterrows():
    feature = {
        "type": "Feature",
        "geometry": {
            "type": "Point",
            "coordinates": [row["lon"], row["lat"]],
        },
        "properties": {
            "times": [row["time"].isoformat()],
            "style": {
                "color": row["color"],
                "fillColor": row["color"],
                "fillOpacity": 0.4,
                "radius": 8
            },
            "icon": "circle"
        }
    }
    features.append(feature)

geojson = {
    "type": "FeatureCollection",
    "features": features
}

# ---------------------------------------------------------
# ■ 地図作成（Folium）
# ---------------------------------------------------------

m = folium.Map(location=[35.3, 135.3], zoom_start=6)

TimestampedGeoJson(
    data=geojson,
    period="PT3H",             # 3時間ごと更新
    duration="PT12H",          # 半日で透明化→消える
    auto_play=True,
    loop=True,
    add_last_point=True
).add_to(m)

# ---------------------------------------------------------
# ■ 保存 & ブラウザ自動起動
# ---------------------------------------------------------

html_file = "map_half_year.html"
m.save(html_file)

time.sleep(0.5)
webbrowser.open(html_file)


C:\Users\asami\AppData\Local\Temp\ipykernel_3540\3549422403.py:16: FutureWarning:

'H' is deprecated and will be removed in a future version, please use 'h' instead.



True

In [151]:
import folium
from folium.plugins import TimestampedGeoJson
import pandas as pd
import numpy as np
import json
import webbrowser
import time

# ---------------------------------------------------------
# ■ 全世界の6時間ごとのデータ生成（サンプル）
# ---------------------------------------------------------

# 1年間の6時間ステップ = 365日 × 4回 = 1460点
times = pd.date_range("2024-01-01", "2024-12-31 18:00", freq="6H")

# 全世界へランダムに点をばらまく
lats = np.random.uniform(-60, 70, len(times))   # 極地すぎると見づらいので制限
lons = np.random.uniform(-180, 180, len(times))

df = pd.DataFrame({
    "time": times,
    "lat": lats,
    "lon": lons
})

# ---------------------------------------------------------
# ■ 色をランダム or 任意ルールで付与（例：緯度で色分け）
# ---------------------------------------------------------
def get_color(lat):
    if lat < -20:
        return "rgba(0,128,255,0.5)"   # 南半球 → 青
    elif lat < 20:
        return "rgba(255,165,0,0.5)"   # 赤道帯 → オレンジ
    else:
        return "rgba(255,0,0,0.5)"     # 北半球 → 赤

df["color"] = df["lat"].apply(get_color)

# ---------------------------------------------------------
# ■ TimeDimension 用 GeoJSON 作成
# ---------------------------------------------------------

features = []
for _, row in df.iterrows():
    features.append({
        "type": "Feature",
        "geometry": {
            "type": "Point",
            "coordinates": [row["lon"], row["lat"]],
        },
        "properties": {
            "times": [row["time"].isoformat()],
            "style": {
                "color": row["color"],
                "fillColor": row["color"],
                "fillOpacity": 0.5,
                "radius": 7
            },
            "icon": "circle"
        }
    })

geo = {
    "type": "FeatureCollection",
    "features": features
}

# ---------------------------------------------------------
# ■ 地図表示（全世界）
# ---------------------------------------------------------

m = folium.Map(
    location=[20, 0],  # 中心はアフリカ上空あたり（世界が見やすい）
    zoom_start=2,
    tiles="cartodbpositron"
)

# ---------------------------------------------------------
# ■ 5秒ごとに更新するアニメーション
# ---------------------------------------------------------

TimestampedGeoJson(
    data=geo,
    period="PT6H",         # ← 6時間刻みデータ
    duration="PT6H",       # ← 6時間後に消える（フェードアウト効果）
    transition_time=5000,  # ← 5秒ごとに次の時刻へ
    auto_play=True,
    loop=True
).add_to(m)

# ---------------------------------------------------------
# ■ 保存 → 自動ブラウザオープン
# ---------------------------------------------------------

html = "global_6h_animation.html"
m.save(html)
time.sleep(0.5)
webbrowser.open(html)


C:\Users\asami\AppData\Local\Temp\ipykernel_3540\2084161809.py:14: FutureWarning:

'H' is deprecated and will be removed in a future version, please use 'h' instead.



True

In [152]:
import folium
from folium.plugins import TimestampedGeoJson
import pandas as pd
import numpy as np
import json
import webbrowser
import time

# ==========================================================
# ■ USGS 過去 30 日の M1.0+ 地震データ取得（世界）
# ==========================================================
url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_month.csv"
df = pd.read_csv(url)

# ==========================================================
# ■ 必要な列を整形
# ==========================================================
df["time"] = pd.to_datetime(df["time"])        # ISO時刻
df["lat"] = df["latitude"]
df["lon"] = df["longitude"]
df["mag"] = df["mag"].fillna(0)
df["depth"] = df["depth"].fillna(0)

# 6時間ごとに丸める（TimeDimension の step と揃える）
df["time_6h"] = df["time"].dt.floor("6H")

# ==========================================================
# ■ 深さで色を決定（浅い→赤 / 深い→青）
# ==========================================================
def depth_to_color(depth):
    # depth: km
    # 0 km → 赤, 700km → 青
    depth_norm = min(max(depth / 700, 0), 1)
    r = int(255 * (1 - depth_norm))
    g = int(80 * (1 - depth_norm))
    b = int(255 * depth_norm)
    return f"rgba({r},{g},{b},0.7)"

df["color"] = df["depth"].apply(depth_to_color)

# 点サイズはマグニチュードを倍率に
df["radius"] = df["mag"].clip(lower=0).apply(lambda m: 2 + m * 2)

# ==========================================================
# ■ GeoJSON の FeatureCollection 生成
# ==========================================================
features = []
for _, row in df.iterrows():
    features.append({
        "type": "Feature",
        "geometry": {
            "type": "Point",
            "coordinates": [row["lon"], row["lat"]],
        },
        "properties": {
            "times": [row["time_6h"].isoformat()],
            "style": {
                "color": row["color"],
                "fillColor": row["color"],
                "fillOpacity": 0.7,
                "radius": row["radius"]
            },
            "icon": "circle"
        }
    })

geo = {
    "type": "FeatureCollection",
    "features": features
}

# ==========================================================
# ■ 地図を作成
# ==========================================================
m = folium.Map(
    location=[20, 0],   # 全世界
    zoom_start=2,
    tiles="cartodbpositron"
)

# ==========================================================
# ■ アニメーション設定
# ==========================================================
TimestampedGeoJson(
    data=geo,

    period="PT6H",           # 6時間刻みで表示
    duration="P1M",          # ★ 1か月経つと点が消える（Fade-out）
    transition_time=2000,    # スライダー更新 2秒
    auto_play=True,
    loop=True
).add_to(m)

# ==========================================================
# ■ 保存＆自動ブラウザ表示
# ==========================================================
html = "eq_animation_month_fade.html"
m.save(html)
time.sleep(0.5)
webbrowser.open(html)


C:\Users\asami\AppData\Local\Temp\ipykernel_3540\129007602.py:25: FutureWarning:

'H' is deprecated and will be removed in a future version, please use 'h' instead.



True

In [153]:
import folium
from folium.plugins import TimestampedGeoJson
import pandas as pd
import numpy as np
import json
import webbrowser
import time

# ==========================================================
# ■ USGS 過去 30 日の M1.0+ 地震データ取得（世界）
# ==========================================================
url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_month.csv"
df = pd.read_csv(url)

# ==========================================================
# ■ 必要な列を整形
# ==========================================================
df["time"] = pd.to_datetime(df["time"])        # ISO時刻
df["lat"] = df["latitude"]
df["lon"] = df["longitude"]
df["mag"] = df["mag"].fillna(0)
df["depth"] = df["depth"].fillna(0)

# 6時間ごとに丸める（TimeDimension の step と揃える）
df["time_6h"] = df["time"].dt.floor("6H")

# ==========================================================
# ■ 深さで色を決定（浅い→赤 / 深い→青）
# ==========================================================
def depth_to_color(depth):
    # depth: km
    # 0 km → 赤, 700km → 青
    depth_norm = min(max(depth / 700, 0), 1)
    r = int(255 * (1 - depth_norm))
    g = int(80 * (1 - depth_norm))
    b = int(255 * depth_norm)
    return f"rgba({r},{g},{b},0.7)"

df["color"] = df["depth"].apply(depth_to_color)

# 点サイズはマグニチュードを倍率に
df["radius"] = df["mag"].clip(lower=0).apply(lambda m: 2 + m * 2)

# ==========================================================
# ■ GeoJSON の FeatureCollection 生成
# ==========================================================
features = []
for _, row in df.iterrows():
    features.append({
        "type": "Feature",
        "geometry": {
            "type": "Point",
            "coordinates": [row["lon"], row["lat"]],
        },
        "properties": {
            "times": [row["time_6h"].isoformat()],
            "style": {
                "color": row["color"],
                "fillColor": row["color"],
                "fillOpacity": 0.7,
                "radius": row["radius"]
            },
            "icon": "circle"
        }
    })

geo = {
    "type": "FeatureCollection",
    "features": features
}

# ==========================================================
# ■ 地図を作成
# ==========================================================
m = folium.Map(
    location=[20, 0],   # 全世界
    zoom_start=2,
    tiles="cartodbpositron"
)

# ==========================================================
# ■ アニメーション設定
# ==========================================================
TimestampedGeoJson(
    data=geo,

    period="PT6H",           # 6時間刻みで表示
    duration="P1M",          # ★ 1か月経つと点が消える（Fade-out）
    transition_time=2000,    # スライダー更新 2秒
    auto_play=True,
    loop=True
).add_to(m)

# ==========================================================
# ■ 保存＆自動ブラウザ表示
# ==========================================================
html = "eq_animation_month_fade.html"
m.save(html)
time.sleep(0.5)
webbrowser.open(html)


C:\Users\asami\AppData\Local\Temp\ipykernel_3540\129007602.py:25: FutureWarning:

'H' is deprecated and will be removed in a future version, please use 'h' instead.



True